# Create Slice for Fabric Rack
This is the slice creation for the ```Exploring Queues``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,7a5adb91-c4c0-4a1c-8021-7b6c56af196f
Bastion Username,lrco247_0000050326
Bastion Private Key File,/home/fabric/work/fabric_config/testingNEWkey
Bastion Host,bastion-1.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
slice_name = "Queues"
site = fablib.get_random_site()

image = "default_ubuntu_20"
nicmodel = "NIC_Basic"

cores = 4
ram = 16
disk = 50

In [3]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    #Router
    router = slice.add_node(name="router", site=site)
    router.set_capacities(cores=cores, ram=ram, disk=disk)
    router.set_image(image)
    rPort1 = router.add_component(model=nicmodel, name="rPort1").get_interfaces()[0] 
    rPort2 = router.add_component(model=nicmodel, name="rPort2").get_interfaces()[0] 
    
    #Host 1
    host1 = slice.add_node(name="Source", site=site)
    host1.set_capacities(cores=cores, ram=ram, disk=disk)
    host1.set_image(image)
    h1Port = host1.add_component(model=nicmodel, name="h1Port").get_interfaces()[0] 
    
    #Host 2
    host2 = slice.add_node(name="Destination", site=site)
    host2.set_capacities(cores=cores, ram=ram, disk=disk)
    host2.set_image(image)
    h2Port = host2.add_component(model=nicmodel, name="h2Port").get_interfaces()[0] 
    
    lan1 = slice.add_l2network(name="Lan1", interfaces=[rPort1, h1Port])
    lan2 = slice.add_l2network(name="Lan2", interfaces=[rPort2, h2Port])
    
    #Submit Slice Request
    slice.submit()
    
except Exception as e:
    print(f"Slice Failed: {e}")


Retry: 8, Time: 264 sec


ID,25d5384c-37b0-43f3-b63c-bbf3abd47ea7
Name,Queues
Lease Expiration (UTC),2023-06-22 20:39:17 +0000
Lease Start (UTC),2023-06-21 20:39:17 +0000
Project ID,7a5adb91-c4c0-4a1c-8021-7b6c56af196f
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
12e298eb-d599-4eb7-99a0-d060b4f37fdb,Destination,4,16,100,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fef2:f53c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fef2:f53c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
346b8fb0-5c3f-47dd-bfb5-8dfe6b43dd42,Source,4,16,100,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fead:845e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fead:845e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0a2a85c8-18e6-4f5f-9994-e2127d9e7c3c,router,4,16,100,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe83:801f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe83:801f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
608f956b-c6b5-41cd-85cd-447b32df821e,Lan1,L2,L2Bridge,CLEM,None,None,Active,
ea5665e1-06e4-4e47-b16d-c9f7467e0f51,Lan2,L2,L2Bridge,CLEM,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
router-rPort2-p1,p1,router,Lan2,100,config,,2A:ED:72:0B:5C:21,ens8,ens8,None,6
router-rPort1-p1,p1,router,Lan1,100,config,,26:EC:72:3C:58:74,ens7,ens7,None,6
Source-h1Port-p1,p1,Source,Lan1,100,config,,2A:F3:42:9A:19:AB,ens7,ens7,None,6
Destination-h2Port-p1,p1,Destination,Lan2,100,config,,2E:46:FE:9A:FE:A2,ens7,ens7,None,6



Time to print interfaces 271 seconds


## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Exploring Queues Notebook, you can ether open it on the explorer or click [Here](./ExploringQueues.ipynb) to open the next notebook.